In [7]:
%pip install autoreject


   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.2/2.7 MB 4.8 MB/s eta 0:00:01
   --- ------------------------------------ 0.2/2.7 MB 4.8 MB/s eta 0:00:01
   ----- ---------------------------------- 0.4/2.7 MB 2.7 MB/s eta 0:00:01
   ------- -------------------------------- 0.5/2.7 MB 2.3 MB/s eta 0:00:01
   -------- ------------------------------- 0.6/2.7 MB 2.2 MB/s eta 0:00:01
   --------- ------------------------------ 0.7/2.7 MB 2.2 MB/s eta 0:00:01
   ----------- ---------------------------- 0.7/2.7 MB 2.1 MB/s eta 0:00:01
   ------------ --------------------------- 0.8/2.7 MB 2.1 MB/s eta 0:00:01
   ------------- -------------------------- 0.9/2.7 MB 2.1 MB/s eta 0:00:01
   -------------- ------------------------- 1.0/2.7 MB 2.1 MB/s eta 0:00:01
   ---------------- ----------------------- 1.1/2.7 MB 2.1 MB/s eta 0:00:01
   ------------------ --------------------- 1.2/2.7 MB 2.1 MB/s eta 0:00:01
   ---------------

In [23]:
# Imports
import os
import pandas as pd
from ipywidgets import *
import numpy as np
import mne
from mne.preprocessing import ICA

# import torch
from mne_icalabel import label_components

import matplotlib.pyplot as plt
from mne.preprocessing import ICA
import scipy
from autoreject import AutoReject, get_rejection_threshold

import preprocessing
import utils

# Specify graph rendering method
# %matplotlib widget
plt.switch_backend("TkAgg")

TMS_EVENT_ID = "Stimulus/S  1"

In [24]:
# Currently for 1 patient, will be generalized into a pipeline for all patients

DATASET_PATH = "./dataset"
FILENAME_TEMPLATE = "TMS-EEG-H_02_S1b_{}_{}.vhdr"

# rsEEG_pre_raw = mne.io.read_raw_brainvision(os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("rsEEG", "pre")), preload=True)
spTEP_pre_raw = mne.io.read_raw_brainvision(
    os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("spTEP", "pre")), preload=True
)
spTEP_pre_raw.drop_channels(["HEOG", "VEOG"])

clone = spTEP_pre_raw.copy()

# rsEEG_post_raw = mne.io.read_raw_brainvision(os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("rsEEG", "post")), preload=True)
# spTEP_post_raw = mne.io.read_raw_brainvision(os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("spTEP", "post")), preload=True)
# spTEP_post_raw.drop_channels(['HEOG', 'VEOG'])

Extracting parameters from ./dataset\TMS-EEG-H_02_S1b_spTEP_pre.vhdr...
Setting channel info structure...
Reading 0 ... 2696199  =      0.000 ...   539.240 secs...


In [3]:
utils.plot_average_response(spTEP_pre_raw, tmin=-0.005, tmax=0.2)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
151 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 151 events and 1026 original time points ...
1 bad epochs dropped


# Pipeline (TESA)

1. EOG
2. Channel rejection (necessary?)
3. Epoching
4. Demeaning
5. TMS-pulse interpolation
6. Downsampling
7. Trials rejection
8. ICA - 1
9. Bandpass & notch filters
10. ICA - 2
11. Bad-channels interpolation
12. Rereferencing
13. Baseline correction


In [8]:
utils.plot_average_response(clone, tmin=-0.005, tmax=0.2)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
151 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 151 events and 1026 original time points ...


1 bad epochs dropped


In [25]:
preprocessing.remove_EOG(clone)

In [26]:
# Epoching
events, event_dict = mne.events_from_annotations(clone)

tms_event_id = event_dict[TMS_EVENT_ID]

epochs = mne.Epochs(
    clone,
    events,
    event_id=event_dict[TMS_EVENT_ID],
    tmin=-1,
    tmax=1,
    baseline=(None, 0),
    reject=None,
    verbose=False,
    detrend=0,
    preload=True,
)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']


In [27]:
epochs = epochs.apply_baseline((None, None))

Applying baseline correction (mode: mean)


In [28]:
tms_window_start = -0.002  # 2ms before
tms_window_end = 0.005     # 5ms after

# Convert time window to samples
sfreq = epochs.info['sfreq']
samples_window_start = int(tms_window_start * sfreq)
samples_window_end = int(tms_window_end * sfreq)

# Iterate over each TMS event
for tms_event in events:
    if tms_event[2] == tms_event_id:  # Check if the event is a TMS pulse event
        # Get the index of the event
        event_index = tms_event[0]

        # Determine the start and end samples of the time window around the TMS pulse
        window_start = event_index + int(tms_window_start * sfreq)
        window_end = event_index + int(tms_window_end * sfreq)

        # Remove the data within the time window around the TMS pulse
        epochs._data[:, :, window_start:window_end] = np.nan

# Cubically interpolate the removed data
epochs.interpolate_bads(reset_bads=True, mode='spline')

Setting channel interpolation method to {'eeg': 'spline'}.


C:\Users\Tomas\AppData\Local\Temp\ipykernel_11184\3460290801.py:23: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  epochs.interpolate_bads(reset_bads=True, mode='spline')


Number of events,150
Events,1: 150
Time range,-1.000 – 1.000 s
Baseline,-1.000 – 1.000 s


In [29]:
print(epochs.info['bads'])  

[]
